In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.9 MB/s eta 0:00:00


In [1]:
from google.colab import drive

In [2]:
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df_train_compressed=pd.read_csv("/content/drive/MyDrive/NLP_project/train_data_compressed_10_percent.csv")

In [10]:
df_train_compressed.head()

article  \
0  By . Mia De Graaf . Britons flocked to beaches...   
1  A couple who weighed a combined 32st were sham...   
2  Video footage shows the heart stopping moment ...   
3  Istanbul, Turkey (CNN) -- About 250 people rac...   
4  By . Daily Mail Reporter . PUBLISHED: . 12:53 ...   

                                             summary  
0  People enjoyed temperatures of 17C at Brighton...  
1  Couple started piling on pounds after the birt...  
2  A 17-year-old boy suffering lacerations to his...  
3  Syrians citizens hightail it to Turkey .\nMost...  
4  The Xue Long had provided the helicopter that ...

In [6]:
df_test_compressed=pd.read_csv("/content/drive/MyDrive/NLP_project/test_data_compressed_10_percent.csv")

In [11]:
!pip install bert-extractive-summarizer

In [12]:
from summarizer import Summarizer

# Example text
text = "Your long piece of text goes here."

# Create a Summarizer object
summarizer = Summarizer()

# Summarize the text
summary = summarizer(text)

# Print the summary
print(summary)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
print(summary)

In [ ]:
# Using first 1000 articles as sample
x_train, y_train = df_train_compressed['article'][:100], df_train_compressed['summary'][:100]
x_test, y_test = df_test_compressed['article'][:100], df_test_compressed['summary'][:100]

In [ ]:
x_train

0     By . Mia De Graaf . Britons flocked to beaches...
1     A couple who weighed a combined 32st were sham...
2     Video footage shows the heart stopping moment ...
3     Istanbul, Turkey (CNN) -- About 250 people rac...
4     By . Daily Mail Reporter . PUBLISHED: . 12:53 ...
                            ...                        
95    A young boy wanders through the high grass, lo...
96    By . Victoria Woollaston . PUBLISHED: . 09:29 ...
97    (CNN) -- All day long, Lucia looked forward to...
98    By . Sam Webb . Fancy living in Margaret Thatc...
99    By . Sam Webb for MailOnline . A father became...
Name: article, Length: 100, dtype: object

In [ ]:
import torch

In [14]:
from summarizer import Summarizer

# Assuming df_test is your DataFrame with 'article' column
test_articles = df_test_compressed['article'].tolist()

# Initialize the BERT Extractive Summarizer
bert_summarizer = Summarizer()

# Apply the BERT model on the test data
test_summaries_bert = [bert_summarizer(article) for article in test_articles]

# Assuming df_test has a 'summary' column
df_test_compressed['bert_summary'] = test_summaries_bert


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [16]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=b9a650a3f29c14d26ad8717b1202aacd655bc4387ce906bb72af71b2d47081c8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [18]:
from rouge_score import rouge_scorer

# Assuming df_test has 'summary' and 'bert_summary' columns
reference_summaries = df_test_compressed['summary'].tolist()
generated_summaries_bert = df_test_compressed['bert_summary'].tolist()

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate ROUGE scores for each pair of reference and BERT-generated summaries
rouge_scores_bert = {
    'rouge1': {'precision': [], 'recall': [], 'fmeasure': []},
    'rouge2': {'precision': [], 'recall': [], 'fmeasure': []},
    'rougeL': {'precision': [], 'recall': [], 'fmeasure': []},
}

for ref, gen_bert in zip(reference_summaries, generated_summaries_bert):
    scores_bert = scorer.score(ref, gen_bert)

    for metric in ['rouge1', 'rouge2', 'rougeL']:
        rouge_scores_bert[metric]['precision'].append(scores_bert[metric].precision)
        rouge_scores_bert[metric]['recall'].append(scores_bert[metric].recall)
        rouge_scores_bert[metric]['fmeasure'].append(scores_bert[metric].fmeasure)

# Calculate average ROUGE scores for BERT-generated summaries
average_rouge_scores_bert = {}
for metric in ['rouge1', 'rouge2', 'rougeL']:
    average_rouge_scores_bert[metric] = {
        'precision': sum(rouge_scores_bert[metric]['precision']) / len(rouge_scores_bert[metric]['precision']),
        'recall': sum(rouge_scores_bert[metric]['recall']) / len(rouge_scores_bert[metric]['recall']),
        'fmeasure': sum(rouge_scores_bert[metric]['fmeasure']) / len(rouge_scores_bert[metric]['fmeasure']),
    }

# Print or use the average ROUGE scores for BERT-generated summaries as needed
print("Average ROUGE Scores for BERT-generated Summaries:")
for metric in ['rouge1', 'rouge2', 'rougeL']:
    print(f"{metric}:")
    print(f"  Precision: {average_rouge_scores_bert[metric]['precision']:.4f}")
    print(f"  Recall: {average_rouge_scores_bert[metric]['recall']:.4f}")
    print(f"  F-Measure: {average_rouge_scores_bert[metric]['fmeasure']:.4f}")


Average ROUGE Scores for BERT-generated Summaries:
rouge1:
  Precision: 0.2263
  Recall: 0.5703
  F-Measure: 0.3088
rouge2:
  Precision: 0.0860
  Recall: 0.2105
  F-Measure: 0.1159
rougeL:
  Precision: 0.1393
  Recall: 0.3508
  F-Measure: 0.1898


In [19]:
print(df_test_compressed['bert_summary'][2])

World No 1 Novak Djokovic has apologised to the startled ball boy caught in the crossfire of a tirade at his support team during his win over Andy Murray in Sunday's Miami Open final. Djokovic lost his cool at the end of the second set as Murray came back to take the match to a decider but has since expressed his regret at the incident in a video posted on Facebook. The world No 1 grabbed a towel from the ball boy (right)  who seemed startled by the loud confrontation . Djokovic then extended his apology to the boy's parents. ' So I want to apologise to his parents for this situation as well. Unfortunately sometimes the emotions get the better of you. '


In [20]:
print(df_test_compressed['summary'][2])

Novak Djokovic beat Andy Murray 7-6 4-6 6-0 in  Miami Open 2015 final .
Djokovic lost his cool after losing the second set to the Brit in Florida .
World No 1 Djokovic shouted at his support team next to a scared ball boy .
After seeing the replay, the Serbian posted an apology video on Facebook .
CLICK HERE for all the latest news from the world of tennis .


In [22]:
comparison_df = pd.DataFrame()

# Add columns for each DataFrame
comparison_df['Original Summary'] = [df_test_compressed['summary'][2]]
comparison_df['BERT Summary'] = [df_test_compressed['bert_summary'][2]]

# Display the comparison DataFrame
print(comparison_df)

                                    Original Summary  \
0  Novak Djokovic beat Andy Murray 7-6 4-6 6-0 in...   

                                        BERT Summary  
0  World No 1 Novak Djokovic has apologised to th...  


In [23]:
from tabulate import tabulate
from IPython.display import display, HTML

# Assuming df is your DataFrame containing summaries
summaries = [
    comparison_df['Original Summary'].iloc[0],
    comparison_df['BERT Summary'].iloc[0],
]

headers = ["Original Summary", "BERT Summary"]
table_data = [summaries]

# Display the table
display(HTML(tabulate(table_data, headers, tablefmt='html')))
